# Neural Network

## Load the Data

In [1]:
import numpy as np
import pandas as pd


train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

## Preprocessing

In [2]:
from preprocessing import preprocessing


X_train, Y_train, X_test = preprocessing(train_data, test_data)

print("X_train.shape =", X_train.shape)
print("Y_train.shape =", Y_train.shape)
print("X_test.shape =", X_test.shape)

X_train.shape = (891, 12)
Y_train.shape = (891,)
X_test.shape = (418, 12)


## Build A Simple Neural Network

### Dataset & DataLoader

In [3]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class TitanicDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.Y is None:
            return self.X[idx, :]
        else:
            return self.X[idx, :], self.Y[idx]

X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train)
X_test = torch.from_numpy(X_test)

train_data = TitanicDataset(X_train, Y_train)

train_dataloader = DataLoader(train_data, batch_size=256, shuffle=True)

### Neural Network

In [4]:
from torch import nn


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
num_features = X_train.shape[1]


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(num_features, int(num_features * 1.25)),
            nn.ReLU(),
            nn.Linear(int(num_features * 1.25), int(num_features * 1.5)),
            nn.ReLU(),
            nn.Linear(int(num_features * 1.5), int(num_features * 2)),
            nn.ReLU(),
            nn.Linear(int(num_features * 2), int(num_features * 1.5)),
            nn.ReLU(),
            nn.Linear(int(num_features * 1.5), int(num_features * 1.25)),
            nn.ReLU(),
            nn.Linear(int(num_features * 1.25), int(num_features * 1)),
            nn.ReLU(),
            nn.Linear(int(num_features * 1), int(num_features * 0.5)),
            nn.ReLU(),
            nn.Linear(int(num_features * 0.5), int(num_features * 0.25)),
            nn.ReLU(),
            nn.Linear(int(num_features * 0.25), 1)
        )
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, X):
        X = self.linear_relu_stack(X)
        Y_hat = self.sigmoid(X)
        
        return Y_hat

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=12, out_features=15, bias=True)
    (1): ReLU()
    (2): Linear(in_features=15, out_features=18, bias=True)
    (3): ReLU()
    (4): Linear(in_features=18, out_features=24, bias=True)
    (5): ReLU()
    (6): Linear(in_features=24, out_features=18, bias=True)
    (7): ReLU()
    (8): Linear(in_features=18, out_features=15, bias=True)
    (9): ReLU()
    (10): Linear(in_features=15, out_features=12, bias=True)
    (11): ReLU()
    (12): Linear(in_features=12, out_features=6, bias=True)
    (13): ReLU()
    (14): Linear(in_features=6, out_features=3, bias=True)
    (15): ReLU()
    (16): Linear(in_features=3, out_features=1, bias=True)
  )
  (sigmoid): Sigmoid()
)


### Optimisation

In [5]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.float()
        y = y.float()
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X).flatten()
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        loss, current = loss.item(), batch * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


epochs = 40
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 0.703206  [    0/  891]
loss: 0.700588  [  256/  891]
loss: 0.700584  [  512/  891]
loss: 0.699867  [  369/  891]
Epoch 2
-------------------------------
loss: 0.701249  [    0/  891]
loss: 0.695804  [  256/  891]
loss: 0.702886  [  512/  891]
loss: 0.701152  [  369/  891]
Epoch 3
-------------------------------
loss: 0.701379  [    0/  891]
loss: 0.697153  [  256/  891]
loss: 0.697744  [  512/  891]
loss: 0.701843  [  369/  891]
Epoch 4
-------------------------------
loss: 0.696667  [    0/  891]
loss: 0.698499  [  256/  891]
loss: 0.699524  [  512/  891]
loss: 0.697881  [  369/  891]
Epoch 5
-------------------------------
loss: 0.697766  [    0/  891]
loss: 0.697798  [  256/  891]
loss: 0.696377  [  512/  891]
loss: 0.697195  [  369/  891]
Epoch 6
-------------------------------
loss: 0.696228  [    0/  891]
loss: 0.696139  [  256/  891]
loss: 0.697150  [  512/  891]
loss: 0.695445  [  369/  891]
Epoch 7
-------------------------------


### Test

In [7]:
def test(model, X_test):
    model.eval()
    with torch.no_grad():
        X_test = X_test.float()
        X_test = X_test.to(device)
        pred = model(X_test).numpy().flatten() >= 0.5
        pred = pred.astype(int)
    return pred

Y_pred = test(model, X_test)

In [8]:
Y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,